In [1]:
from google.colab import files
uploaded = files.upload()  # Click "Choose Files" and select your products.csv

Saving combined_stock_features.csv to combined_stock_features.csv


In [16]:
stock_data = pd.read_csv(io.BytesIO(uploaded['combined_stock_features.csv']))
print(stock_data)

             Date       Close        High         Low        Open  \
0      2023-01-03  123.632530  129.395518  122.742873  128.782649   
1      2023-01-04  124.907692  127.181260  123.642405  125.431599   
2      2023-01-05  123.583099  126.301493  123.326093  125.668849   
3      2023-01-06  128.130203  128.792501  123.454572  124.561702   
4      2023-01-09  128.654129  131.876670  128.397123  128.970458   
...           ...         ...         ...         ...         ...   
24495  2023-12-22   97.674706   98.661898   97.588442   98.058076   
24496  2023-12-26   97.895149   98.748161   97.875984   98.470212   
24497  2023-12-27   97.435089   98.288101   97.128381   97.799294   
24498  2023-12-28   96.026192   97.387176   95.968680   97.176318   
24499  2023-12-29   95.824921   96.409563   95.479881   96.198712   

            Volume        MA10        MA50       MA100       MA200        RSI  \
0      112117500.0  128.989244  141.020710  146.761524  149.348867  22.616920   
1       8

In [2]:
import pandas as pd
import io

# Read the uploaded file
df = pd.read_csv(io.BytesIO(uploaded['combined_stock_features.csv']))
print(df)

# Or if you just want to save it first:
with open('combined_stock_features.csv', 'wb') as f:
    f.write(uploaded['combined_stock_features.csv'])

df = pd.read_csv('combined_stock_features.csv')
print(df)

             Date       Close        High         Low        Open  \
0      2023-01-03  123.632530  129.395518  122.742873  128.782649   
1      2023-01-04  124.907692  127.181260  123.642405  125.431599   
2      2023-01-05  123.583099  126.301493  123.326093  125.668849   
3      2023-01-06  128.130203  128.792501  123.454572  124.561702   
4      2023-01-09  128.654129  131.876670  128.397123  128.970458   
...           ...         ...         ...         ...         ...   
24495  2023-12-22   97.674706   98.661898   97.588442   98.058076   
24496  2023-12-26   97.895149   98.748161   97.875984   98.470212   
24497  2023-12-27   97.435089   98.288101   97.128381   97.799294   
24498  2023-12-28   96.026192   97.387176   95.968680   97.176318   
24499  2023-12-29   95.824921   96.409563   95.479881   96.198712   

            Volume        MA10        MA50       MA100       MA200        RSI  \
0      112117500.0  128.989244  141.020710  146.761524  149.348867  22.616920   
1       8

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# # Load the enhanced dataset
# def load_enhanced_data(ticker="MSFT"):
#     df = pd.read_csv(f"{ticker}_stock_features.csv")
#     df['Date'] = pd.to_datetime(df['Date'])  # Ensure Date is datetime
#     df.set_index('Date', inplace=True)
#     return df

# ticker = "MSFT"
# data = load_enhanced_data(ticker)
# print(data.head())  # Verify all features are loaded

In [4]:
ticker_to_id = {ticker: idx for idx, ticker in enumerate(df['Ticker'].unique())}
df['Ticker_ID'] = df['Ticker'].map(ticker_to_id)

In [5]:
# Select features and target
features = ['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper', 'Bollinger_Lower', 'Log_Returns', 'Ticker_ID']
data = df[features]  # Keep only selected features

In [6]:
data

,Close,MA100,RSI,Norm_Volume,Bollinger_Upper,Bollinger_Lower,Log_Returns,Ticker_ID
0,123.632530,146.761524,22.616920,1.331549,148.054166,121.250634,-0.038122,0
1,124.907692,146.347825,23.287572,1.057702,146.806195,120.494900,0.010261,0
2,123.583099,145.885253,23.902332,0.959823,146.141766,119.390889,-0.010661,0
3,128.130203,145.457396,39.492908,1.037141,145.362310,119.051351,0.036133,0
4,128.654129,145.036356,43.030678,0.840106,144.150865,119.027163,0.004081,0
...,...,...,...,...,...,...,...,...
24495,97.674706,103.023651,48.316058,0.588664,100.445159,93.591180,0.001768,97
24496,97.895149,102.993496,56.213483,0.776447,100.160122,93.701782,0.002254,97
24497,97.435089,102.955914,59.937609,0.673882,99.815752,93.831461,-0.004711,97
24498,96.026192,102.906316,56.502614,0.758253,99.671196,93.769953,-0.014565,97


In [7]:
def min_max_scaling(data):
    """Scale data to [0, 1] range and return scaled data + min/max values"""
    min_val = np.min(data)
    max_val = np.max(data)
    # Handle case where all values are identical (avoid division by zero)
    if max_val == min_val:
        return np.zeros_like(data), min_val, max_val
    return (data - min_val) / (max_val - min_val), min_val, max_val

In [8]:
def scale_features(df):
    """Scale each feature column independently"""
    scaled_data = {}
    scalers = {}  # To store min/max for each feature

    for col in df.columns:
        if col == 'Ticker_ID':
            scaled_data[col] = df[col].values  # Keep as is (integer IDs)
            continue

        scaled_values, min_val, max_val = min_max_scaling(df[col].values)
        if col == 'Log_Returns':
            scalers['target'] = (min_val, max_val)
        else:
            scalers[col] = (min_val, max_val)

        scaled_data[col] = scaled_values

    return pd.DataFrame(scaled_data), scalers

# Scale all features
scaled_data, scalers = scale_features(data)

# Split into train/test (keeping temporal order)
split_ratio = 0.8
split_index = int(len(scaled_data) * split_ratio)
train_data = scaled_data.iloc[:split_index]
test_data = scaled_data.iloc[split_index:]

In [9]:
print(train_data.columns)
print(test_data.columns)

Index(['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper',
       'Bollinger_Lower', 'Log_Returns', 'Ticker_ID'],
      dtype='object')
Index(['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper',
       'Bollinger_Lower', 'Log_Returns', 'Ticker_ID'],
      dtype='object')


In [10]:
# Create ticker mapping and ticker IDs
unique_tickers = list(set(train_data['Ticker_ID'].values) | set(test_data['Ticker_ID'].values))
ticker_mapping = {ticker: idx for idx, ticker in enumerate(unique_tickers)}

train_ticker_ids = train_data['Ticker_ID'].values
test_ticker_ids = test_data['Ticker_ID'].values

In [12]:
def create_sequences_multi(data, ticker_ids, seq_length=50, target_col='Log_Returns'):
    """Create sequences from multiple features (without including Ticker_ID)"""
    X, y, ticker_seq_list = [], [], []
    data_values = data.drop(['Ticker_ID'], axis=1).values  # Drop 'Ticker_ID' from features

    for ticker_id in np.unique(ticker_ids):
        ticker_data = data[data['Ticker_ID'] == ticker_id]
        features = ticker_data.drop(['Log_Returns', 'Ticker_ID'], axis=1).values
        targets = ticker_data['Log_Returns'].values

        for i in range(len(features) - seq_length):
            # Create a sequence of features (X)
            X.append(features[i:i+seq_length, :])  # Selecting only the features
            # Append the corresponding ticker_id to the ticker_seq_list
            ticker_seq_list.append(ticker_id)
            # The target is the Log_Returns for the next time step
            y.append(targets[i + seq_length])

    return np.array(X), np.array(y), np.array(ticker_seq_list)

seq_length = 50

# After fixing the sequence generation, create the sequences for train and test data
X_train, y_train, train_tickers = create_sequences_multi(train_data, train_ticker_ids, seq_length)
X_test, y_test, test_tickers = create_sequences_multi(test_data, test_ticker_ids, seq_length)

# Check shapes
print(f"Training shapes - X: {X_train.shape}, y: {y_train.shape}")
print(f"Test shapes - X: {X_test.shape}, y: {y_test.shape}")


Training shapes - X: (15650, 50, 6), y: (15650,)
Test shapes - X: (3900, 50, 6), y: (3900,)


In [13]:
# Add Adam optimizer support in LSTM class

class LSTM:
    def __init__(self, input_dim, hidden_dim, output_dim, ticker_dim, embedding_dim, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lr = learning_rate
        self.ticker_dim = ticker_dim
        self.embedding_dim = embedding_dim
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.t = 1  # time step for Adam

        # Combined input size: hidden + input + ticker embedding
        concat_dim = hidden_dim + input_dim + embedding_dim

        # Initialize weights
        self.Wf = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bf = np.zeros((hidden_dim, 1))

        self.Wi = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bi = np.zeros((hidden_dim, 1))

        self.Wc = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bc = np.zeros((hidden_dim, 1))

        self.Wo = np.random.randn(hidden_dim, concat_dim) * 0.01
        self.bo = np.zeros((hidden_dim, 1))

        # Output layer
        self.Wy = np.random.randn(output_dim, hidden_dim) * 0.01
        self.by = np.zeros((output_dim, 1))

        # Initialize embedding matrix for Ticker_ID
        self.ticker_embedding = np.random.randn(self.ticker_dim, self.embedding_dim) * 0.01

        # Initialize Adam moment estimates
        self._init_adam_params()

    def _init_adam_params(self):
        self.m = {}
        self.v = {}
        for param_name in ['Wf', 'Wi', 'Wc', 'Wo', 'Wy', 'bf', 'bi', 'bc', 'bo', 'by']:
            param = getattr(self, param_name)
            self.m[param_name] = np.zeros_like(param)
            self.v[param_name] = np.zeros_like(param)

    def get_ticker_embedding(self, ticker_id):
        """Return the embedding for the ticker ID"""
        return self.ticker_embedding[ticker_id].reshape(-1, 1)

    def sigmoid(self, x): return 1 / (1 + np.exp(-x))
    def dsigmoid(self, x): return x * (1 - x)
    def tanh(self, x): return np.tanh(x)
    def dtanh(self, x): return 1 - x ** 2


    def forward(self, x_seq, ticker_id, h=None, c=None):

      if h is None:
          h = np.zeros((self.hidden_dim, 1))
      if c is None:
          c = np.zeros((self.hidden_dim, 1))
      self.caches = []

    # Get embedding once per sequence (not per timestep)
      ticker_embedding = self.get_ticker_embedding(ticker_id).reshape(-1, 1)

      for x in x_seq:
          x = x.reshape(self.input_dim, 1)

          # Concatenate previous hidden state, input, and ticker embedding
          concat = np.vstack((h, x, ticker_embedding))

          ft = self.sigmoid(np.dot(self.Wf, concat) + self.bf)
          it = self.sigmoid(np.dot(self.Wi, concat) + self.bi)
          c_tilde = self.tanh(np.dot(self.Wc, concat) + self.bc)
          c = ft * c + it * c_tilde
          ot = self.sigmoid(np.dot(self.Wo, concat) + self.bo)
          h = ot * self.tanh(c)

          self.caches.append((h, c, ft, it, c_tilde, ot, concat))

      y_hat = np.dot(self.Wy, h) + self.by
      return y_hat, h, c

    def backward(self, x_seq, y_hat, y_true):
        dh_next = np.zeros((self.hidden_dim, 1))
        dc_next = np.zeros((self.hidden_dim, 1))

        grads = {
            'Wf': np.zeros_like(self.Wf), 'Wi': np.zeros_like(self.Wi),
            'Wc': np.zeros_like(self.Wc), 'Wo': np.zeros_like(self.Wo),
            'Wy': np.zeros_like(self.Wy),
            'bf': np.zeros_like(self.bf), 'bi': np.zeros_like(self.bi),
            'bc': np.zeros_like(self.bc), 'bo': np.zeros_like(self.bo),
            'by': np.zeros_like(self.by)
        }

        dy = y_hat - y_true
        grads['Wy'] += np.dot(dy, self.caches[-1][0].T)
        grads['by'] += dy

        dh = np.dot(self.Wy.T, dy) + dh_next

        for t in reversed(range(len(x_seq))):
            h, c, ft, it, c_tilde, ot, concat = self.caches[t]
            c_prev = self.caches[t - 1][1] if t > 0 else np.zeros_like(c)

            do = dh * self.tanh(c)
            do_raw = do * self.dsigmoid(ot)

            dc = dh * ot * self.dtanh(self.tanh(c)) + dc_next
            dc_tilde = dc * it
            dc_tilde_raw = dc_tilde * self.dtanh(c_tilde)

            di = dc * c_tilde
            di_raw = di * self.dsigmoid(it)

            df = dc * c_prev
            df_raw = df * self.dsigmoid(ft)

            grads['Wf'] += np.dot(df_raw, concat.T)
            grads['Wi'] += np.dot(di_raw, concat.T)
            grads['Wc'] += np.dot(dc_tilde_raw, concat.T)
            grads['Wo'] += np.dot(do_raw, concat.T)

            grads['bf'] += df_raw
            grads['bi'] += di_raw
            grads['bc'] += dc_tilde_raw
            grads['bo'] += do_raw

            dconcat = (np.dot(self.Wf.T, df_raw) +
                       np.dot(self.Wi.T, di_raw) +
                       np.dot(self.Wc.T, dc_tilde_raw) +
                       np.dot(self.Wo.T, do_raw))

            dh = dconcat[:self.hidden_dim, :]
            dc_next = dc * ft

        self._apply_adam(grads)
        self.t += 1  # Increment timestep

    def _apply_adam(self, grads):
        for param_name in grads:
            grad = grads[param_name]
            self.m[param_name] = self.beta1 * self.m[param_name] + (1 - self.beta1) * grad
            self.v[param_name] = self.beta2 * self.v[param_name] + (1 - self.beta2) * (grad ** 2)

            m_hat = self.m[param_name] / (1 - self.beta1 ** self.t)
            v_hat = self.v[param_name] / (1 - self.beta2 ** self.t)

            param = getattr(self, param_name)
            param -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            setattr(self, param_name, param)

    def train(self, X_train, y_train, ticker_ids_train, epochs=10, batch_size=32):
        for epoch in range(epochs):
            total_loss = 0
            indices = np.arange(len(X_train))
            np.random.shuffle(indices)

            for i in range(0, len(X_train), batch_size):
                batch_indices = indices[i:i+batch_size]
                batch_loss = 0
                batch_grads = []

                for j in batch_indices:
                    x_seq = X_train[j]
                    y_true = y_train[j].reshape(self.output_dim, 1)
                    ticker_ids = ticker_ids_train[j]
                    y_hat, _, _ = self.forward(x_seq, ticker_ids)
                    loss = np.mean((y_hat - y_true) ** 2)
                    batch_loss += loss

                    self.backward(x_seq, y_hat, y_true)

                total_loss += batch_loss

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.6f}")


In [14]:
X_train, y_train, train_tickers = create_sequences_multi(train_data, train_data['Ticker_ID'].values, seq_length)
X_test, y_test, test_tickers = create_sequences_multi(test_data, test_data['Ticker_ID'].values, seq_length)

lstm = LSTM(input_dim=X_train.shape[2], hidden_dim=128, output_dim=1,
            ticker_dim=len(ticker_mapping), embedding_dim=32, learning_rate=0.001)

lstm.train(X_train, y_train, train_tickers, epochs=50, batch_size=64)


Epoch 1/50, Loss: 38.221419
Epoch 2/50, Loss: 29.036953
Epoch 3/50, Loss: 28.631045
Epoch 4/50, Loss: 28.589570
Epoch 5/50, Loss: 28.514351
Epoch 6/50, Loss: 28.494187
Epoch 7/50, Loss: 28.482345
Epoch 8/50, Loss: 28.413927
Epoch 9/50, Loss: 28.427862
Epoch 10/50, Loss: 28.398818
Epoch 11/50, Loss: 28.481623
Epoch 12/50, Loss: 28.451918
Epoch 13/50, Loss: 28.333800
Epoch 14/50, Loss: 28.197135
Epoch 15/50, Loss: 28.253409
Epoch 16/50, Loss: 28.151639
Epoch 17/50, Loss: 28.226775
Epoch 18/50, Loss: 28.170987
Epoch 19/50, Loss: 28.167808
Epoch 20/50, Loss: 28.147758
Epoch 21/50, Loss: 28.065150
Epoch 22/50, Loss: 28.073832
Epoch 23/50, Loss: 28.031162
Epoch 24/50, Loss: 28.078044
Epoch 25/50, Loss: 28.114154
Epoch 26/50, Loss: 28.375393
Epoch 27/50, Loss: 28.335240
Epoch 28/50, Loss: 29.629784
Epoch 29/50, Loss: 29.393289
Epoch 30/50, Loss: 29.954902
Epoch 31/50, Loss: 29.352160
Epoch 32/50, Loss: 30.743129
Epoch 33/50, Loss: 30.259613
Epoch 34/50, Loss: 31.733701
Epoch 35/50, Loss: 31.3

In [15]:
def predict_multi(lstm, X_test, test_tickers, scalers):
    predictions = []

    for i in range(len(X_test)):
        x_seq = X_test[i]
        ticker_ids = test_tickers[i]
        # Reinitialize h and c for stateless predictions
        h_prev = np.zeros((lstm.hidden_dim, 1))
        c_prev = np.zeros((lstm.hidden_dim, 1))
        y_pred, _, _ = lstm.forward(x_seq, ticker_ids, h_prev, c_prev)
        predictions.append(y_pred.flatten()[0])

    # Inverse scale predictions
    min_target, max_target = scalers['target']
    predictions = np.array(predictions) * (max_target - min_target) + min_target
    return predictions


In [21]:
def predict_for_stock(lstm, stock_df, ticker, ticker_mapping, scalers, seq_length):
    """
    Predict target for a specific stock using the last available sequence.
    """
    # Get ticker ID from ticker_mapping
    ticker_id = ticker_mapping[ticker]

    # Merge stock_df with data to get the Ticker_ID
    stock_df_with_ticker_id = stock_df.copy()  # Avoid modifying the original stock_df
    stock_df_with_ticker_id['Ticker_ID'] = stock_df_with_ticker_id['Ticker'].map(ticker_mapping)

    # Filter data for the specific stock
    stock_data_filtered = stock_df_with_ticker_id[stock_df_with_ticker_id['Ticker'] == ticker]

    # Ensure there's enough data for the sequence
    if len(stock_data_filtered) < seq_length:
        raise ValueError(f"Not enough data for stock {ticker}. Need at least {seq_length} rows.")

    # Select only the features that the model expects (6 features)
    required_columns = ['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper', 'Bollinger_Lower']
    x_seq = stock_data_filtered[required_columns].iloc[-seq_length:].values

    # Debugging: Check the shape of x_seq
    print(f"Shape of x_seq: {x_seq.shape}")

    # Ensure x_seq has the correct number of features (input_dim)
    if x_seq.shape[1] != lstm.input_dim:
        raise ValueError(f"Shape mismatch! x_seq has {x_seq.shape[1]} features, but model expects {lstm.input_dim} features.")

    # Forward pass
    h_prev = np.zeros((lstm.hidden_dim, 1))
    c_prev = np.zeros((lstm.hidden_dim, 1))
    y_pred, _, _ = lstm.forward(x_seq, ticker_id, h_prev, c_prev)

    # Inverse scale
    min_target, max_target = scalers['target']
    prediction = y_pred.flatten()[0] * (max_target - min_target) + min_target

    return prediction


In [17]:
stock_data.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'MA10', 'MA50',
       'MA100', 'MA200', 'RSI', 'Norm_Volume', 'Bollinger_Upper',
       'Bollinger_Lower', 'Log_Returns', 'Pct_Change', 'Ticker'],
      dtype='object')

In [18]:
data.columns

Index(['Close', 'MA100', 'RSI', 'Norm_Volume', 'Bollinger_Upper',
       'Bollinger_Lower', 'Log_Returns', 'Ticker_ID'],
      dtype='object')

In [22]:
#Create a reverse mapping from Ticker to Ticker_ID using stock_data and data
ticker_mapping = dict(zip(stock_data['Ticker'], data['Ticker_ID']))

# Now, when you want to predict for a specific stock (e.g., 'AAPL')
ticker = 'AAPL'

# Get the Ticker_ID corresponding to the ticker (e.g., 'AAPL')
ticker_id = ticker_mapping[ticker]

# Now call the predict_for_stock function using the ticker symbol (not the ticker_id)
predicted_value = predict_for_stock(lstm, stock_data, ticker, ticker_mapping, scalers, seq_length)

# Print the predicted log_return/volatility for 'AAPL'
print(f"Predicted log_return/volatility for {ticker}: {predicted_value}")

Shape of x_seq: (50, 6)
Predicted log_return/volatility for AAPL: -0.031000607942739275


<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))


In [23]:
def predict_for_stock_over_time(lstm, stock_df, ticker, ticker_mapping, scalers, seq_length):
    """
    Predict target (log_return/volatility) for every time step after the first seq_length
    for a specific stock. Returns a list of predictions.
    """
    # Get Ticker ID
    ticker_id = ticker_mapping[ticker]

    # Filter data for the stock
    stock_data = stock_df[stock_df['Ticker'] == ticker].copy()

    # Ensure enough data
    if len(stock_data) < seq_length:
        raise ValueError(f"Not enough data for stock {ticker}. Need at least {seq_length} rows.")

    # Features used for input
    feature_cols = ['Close', 'MA100', 'RSI', 'Norm_Volume',
                    'Bollinger_Upper', 'Bollinger_Lower']  # Input_dim = 6

    # Reset hidden state (optional: you can also keep it flowing if desired)
    h_prev = np.zeros((lstm.hidden_dim, 1))
    c_prev = np.zeros((lstm.hidden_dim, 1))

    predictions = []

    # Loop over all possible sequences
    for i in range(len(stock_data) - seq_length):
        x_seq = stock_data.iloc[i:i+seq_length][feature_cols].values

        if x_seq.shape[1] != lstm.input_dim:
            raise ValueError(f"Shape mismatch! x_seq has {x_seq.shape[1]} features, but model expects {lstm.input_dim} features.")

        # Forward pass
        y_pred, h_prev, c_prev = lstm.forward(x_seq, ticker_id, h_prev, c_prev)

        # Inverse scale the prediction
        min_target, max_target = scalers['target']
        prediction = y_pred.flatten()[0] * (max_target - min_target) + min_target
        predictions.append(prediction)

    return predictions


In [24]:
predicted_values = predict_for_stock_over_time(lstm, stock_data, 'AAPL', ticker_mapping, scalers, seq_length)
print(f"Predicted log_returns for AAPL ({len(predicted_values)} values):")
print(predicted_values)

<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in 

Predicted log_returns for AAPL (200 values):
[np.float64(0.003613416253128443), np.float64(-0.0065511731036390075), np.float64(-0.005656367856616329), np.float64(-0.006548961091080496), np.float64(-0.006559913643041038), np.float64(-0.006559311305655763), np.float64(-0.006392615241235244), np.float64(-0.004459280259005605), np.float64(-0.005417556276478441), np.float64(0.0029367378480914808), np.float64(-0.006559581185714175), np.float64(-0.006559918164495471), np.float64(-0.006559918193116521), np.float64(-0.006559918069177967), np.float64(-0.006559915190661875), np.float64(-0.006406858324122466), np.float64(-0.006557289869332877), np.float64(0.003662314595065441), np.float64(-0.03111145719810947), np.float64(-0.05492202543099038), np.float64(0.0002644072780427775), np.float64(0.003679327179180708), np.float64(-0.003196195831293047), np.float64(-0.006406541259938114), np.float64(-0.0026146209902812023), np.float64(0.0036247793312026155), np.float64(-0.031000608846521935), np.float64(-

<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))


In [25]:
import pandas as pd

# 1. Create an empty list to collect results
all_predictions = []

# 2. Loop over all tickers
for ticker in stock_data['Ticker'].unique():
    try:
        # Predict for this stock
        preds = predict_for_stock_over_time(lstm, stock_data, ticker, ticker_mapping, scalers, seq_length)

        # Get corresponding dates (starting from seq_length-th date)
        stock_dates = stock_data[stock_data['Ticker'] == ticker].iloc[seq_length:]['Date'].values

        # Combine into rows
        for date, pred in zip(stock_dates, preds):
            all_predictions.append({
                'Date': date,
                'Ticker': ticker,
                'Predicted_Log_Return': pred
            })

    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")

# 3. Create a DataFrame
predictions_df = pd.DataFrame(all_predictions)

# Sort by Ticker first, then Date
predictions_df = predictions_df.sort_values(['Ticker', 'Date'])

# Save to CSV
predictions_df.to_csv('predicted_log_returns.csv', index=False)

print("✅ Saved all stock predictions sorted by Ticker and Date!")


Streaming output truncated to the last 5000 lines.
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d686e642>:54: RuntimeWarning: overflow encountered in exp
  def sigmoid(self, x): return 1 / (1 + np.exp(-x))
<ipython-input-13-2705d68

✅ Saved all stock predictions to 'all_stocks_predicted_log_returns.csv'


In [28]:
window_size = 20  # you can change to 10, 30, etc.

predictions_df['Predicted_Volatility'] = (
    predictions_df
    .groupby('Ticker')['Predicted_Log_Return']
    .rolling(window=window_size)
    .std()
    .reset_index(level=0, drop=True)
)

# Optional: sort again if needed
predictions_df = predictions_df.sort_values(['Ticker', 'Date'])

# Save volatility predictions
predictions_df[['Date', 'Ticker', 'Predicted_Volatility']].to_csv('all_stocks_predicted_volatility.csv', index=False)

print("✅ Volatility predictions saved!")


✅ Volatility predictions saved!
